In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab2RddApp") \
    .getOrCreate()

sc = spark.sparkContext

## Partie 1 : Word count dans un fichier txt.

Charger le fichier moby_dick.txt dans un RDD.

In [2]:
moby_rdd = sc.textFile("../../data/moby_dick.txt")

In [3]:
!python --version

Python 3.11.6


Compter et afficher le nombre de lignes de celui-ci.

In [4]:
print(moby_rdd.count())

21933


Créer un nouveau RDD qui contiendra les lignes contenant le mot ‘chapter’ (non sensible à la casse). Combien y en a-t-il ?

In [5]:
chapter_rdd = moby_rdd.filter(lambda x: "chapter" in x.lower())
print(chapter_rdd.count())

316


Créer un nouveau RDD qui contiendra uniquement les lignes non vides de mobyDick. Combien y en a-t-il ?


In [6]:
non_empty_rdd = moby_rdd.filter(lambda x: len(x.strip()) > 0)
print(non_empty_rdd.count())

18913


Compter le nombre d’occurrences de chaque mot dans le document obtenu à la question précédente, en suivant l’approche du WordCount de MapReduce. Consulter la documentation des méthodes flatMap, map et reduceByKey. Combien y a-t-il de mots différents au total ?

In [7]:
non_empty_rdd.flatMap(lambda x: x.split()).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

PythonRDD[9] at RDD at PythonRDD.scala:53

In [8]:
word_counts = non_empty_rdd.flatMap(str.split).map(lambda x: (x, 1)).reduceByKey(int.__add__)

In [9]:
print(word_counts.count())

33086


Afficher les 10 mots les plus fréquents du livre.

In [10]:
word_counts.map(lambda x: (x[1], x[0])).top(10)

[(13694, 'the'),
 (6531, 'of'),
 (5932, 'and'),
 (4493, 'a'),
 (4459, 'to'),
 (3850, 'in'),
 (2679, 'that'),
 (2428, 'his'),
 (1723, 'I'),
 (1649, 'with')]

## Partie 2 : Représentation d'un graphe sous forme de RDD (sans utiliser GraphX ni de GraphFrame).

On considère l’ensemble de triplets suivants, représentant les arcs d'un graphe :

In [11]:
graph = {
    (1, 0, 5),
    (5, 1, 8),
    (8, 2, 1),
    (2 ,0 ,6),
    (3, 0, 6),
    (6, 1, 9),
    (5, 1, 9),
    (9, 3, 11),
    (9, 4, 12),
    (4, 0, 7),
    (7, 1, 9),
    (7, 2, 10),
    (14, 1, 15),
    (15, 1, 16),
    (14, 1, 16),
    (17, 0, 18),
    (18, 0, 19),
    (19, 1, 20),
    (20, 0, 17),
}

La structure de chaque arc est la suivante :
    <br>- le premier élément correspond à l’identifiant du nœud d'origine (sujet du triplet),
    <br>- le second élément correspond au label de l’arc,
    <br>- le troisième et dernier élément correspond à l’identifiant du nœud d'arrivée (objet du triplet).

Charger le graphe dans un RDD. Compter et afficher le nombre d'arcs qu'il contient.

In [12]:
triplet_rdd = sc.parallelize(graph)
triplet_rdd.collect()
print(triplet_rdd.count())

19


En pratique nous ne souhaitons pas utiliser les labels des arcs. Créer un nouveau RDD ne comportant que les paires (sujet, objet).

In [13]:
so_pair_rdd = triplet_rdd.map(lambda x: (x[0], x[2]))

On appelle racine un noeud qui ne reçoit aucun arc (qui n'est objet d'aucun triplet). Créer un RDD ne contenant que les racines du graphe. Pour cela, créer deux rdd intermédiaires contenant l'ensemble des objets d'une part et des sujets d'autre part et les persister au niveau MEMORY_ONLY. Les racines sont les sujets privés des objets.
Afficher les racines.

In [14]:
from pyspark import StorageLevel

In [15]:
subjects = so_pair_rdd.map(lambda x: x[0]).distinct().persist(storageLevel=StorageLevel.MEMORY_ONLY)
objects = so_pair_rdd.map(lambda x: x[1]).distinct().persist(storageLevel=StorageLevel.MEMORY_ONLY)

In [16]:
roots = subjects.subtract(objects)
roots.collect()

[2, 3, 4, 14]

De manière analogue, calculer et afficher les feuilles, noeuds qui ne sont origines (sujets) d'aucun arc.

In [17]:
leaves = objects.subtract(subjects)
leaves.collect()

[16, 10, 11, 12]

Question optionnelle, plus difficile :
Créer un nouveau RDD qui contient la "fermeture transitive" du graphe, correspondant à l'ensemble des paires (origine, destination) où la destination est accessible à partir de l'origine à partir d'un chemin composé d'un ou plusieurs arc.
Par exemple, si le graphe était {(1, 2), (2, 3)}, alors sa fermeture transitive serait {(1, 2), (2, 3), (1, 3)}, car 2 est accessible directement à partir de 1, 3 est accessible directement à partir de 2, et 3 est accessible indirectement à partir de 1, en deux étapes.


In [19]:
so_pair_rdd.map(
    lambda x: (x[1], x[0])
).join(
    so_pair_rdd
).map(
    lambda x: x[1]
).collect()

[(1, (8, 5)),
 (17, (20, 18)),
 (18, (17, 19)),
 (19, (18, 20)),
 (20, (19, 17)),
 (5, (1, 9)),
 (5, (1, 8)),
 (6, (2, 9)),
 (6, (3, 9)),
 (7, (4, 10)),
 (7, (4, 9)),
 (8, (5, 1)),
 (9, (5, 11)),
 (9, (5, 12)),
 (9, (7, 11)),
 (9, (7, 12)),
 (9, (6, 11)),
 (9, (6, 12)),
 (15, (14, 16))]

In [21]:
def transitive_closure(so_pair_rdd):
    
    def loop(old_pair_rdd, old_size):
        new_pair_rdd = old_pair_rdd.map(
            lambda x: (x[1], x[0])
        ).join(
            old_pair_rdd
        ).map(
            lambda x: x[1]
        ).union(
            old_pair_rdd
        ).distinct()
        new_size = new_pair_rdd.count()
        if old_size != new_size:
            return loop(new_pair_rdd, new_size)
        return new_pair_rdd
    
    return loop(so_pair_rdd, so_pair_rdd.count())

In [22]:
tc_rdd = transitive_closure(so_pair_rdd)
tc_rdd.collect()

[(20, 20),
 (7, 9),
 (1, 5),
 (1, 11),
 (4, 12),
 (14, 16),
 (5, 8),
 (2, 9),
 (7, 12),
 (5, 11),
 (8, 11),
 (20, 17),
 (17, 20),
 (8, 8),
 (4, 9),
 (9, 12),
 (5, 1),
 (8, 5),
 (7, 11),
 (1, 9),
 (17, 18),
 (5, 9),
 (19, 18),
 (18, 19),
 (1, 1),
 (19, 17),
 (5, 12),
 (18, 20),
 (15, 16),
 (20, 19),
 (3, 9),
 (1, 12),
 (4, 11),
 (6, 11),
 (5, 5),
 (19, 20),
 (8, 1),
 (17, 19),
 (2, 6),
 (3, 12),
 (18, 17),
 (9, 11),
 (19, 19),
 (2, 11),
 (6, 12),
 (3, 11),
 (8, 12),
 (18, 18),
 (4, 7),
 (2, 12),
 (14, 15),
 (20, 18),
 (8, 9),
 (6, 9),
 (3, 6),
 (4, 10),
 (7, 10),
 (1, 8),
 (17, 17)]

Déterminer et afficher les paires (sujet, objet) qui ont été ajoutées dans la fermeture transitive, en les triant par ordre croissant de sujet puis objet.

In [20]:
tc_rdd.subtract(so_pair_rdd).collect()

[(7, 11),
 (20, 19),
 (3, 9),
 (8, 12),
 (17, 17),
 (1, 9),
 (8, 9),
 (4, 12),
 (2, 11),
 (5, 12),
 (18, 20),
 (4, 11),
 (2, 12),
 (6, 11),
 (19, 19),
 (1, 8),
 (7, 12),
 (5, 5),
 (3, 12),
 (5, 11),
 (17, 19),
 (1, 1),
 (1, 11),
 (3, 11),
 (17, 20),
 (4, 9),
 (1, 12),
 (19, 18),
 (5, 1),
 (20, 18),
 (18, 17),
 (19, 17),
 (8, 11),
 (2, 9),
 (6, 12),
 (18, 18),
 (8, 8),
 (20, 20),
 (4, 10),
 (8, 5)]

Créer un RDD contenant l'ensemble des nœuds accessibles à partir d'une racine.
Chaque élément de ce RDD contiendra un tuple avec la racine en première position et une liste triée de tous les nœuds accessibles en deuxième position.

In [25]:
tc_rdd.groupByKey().collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x726481b38950>),
 (2, <pyspark.resultiterable.ResultIterable at 0x726481b50c90>),
 (3, <pyspark.resultiterable.ResultIterable at 0x726481b53990>),
 (4, <pyspark.resultiterable.ResultIterable at 0x726481b7b8d0>),
 (5, <pyspark.resultiterable.ResultIterable at 0x726481b50050>),
 (6, <pyspark.resultiterable.ResultIterable at 0x726481b38990>),
 (7, <pyspark.resultiterable.ResultIterable at 0x726433c03010>),
 (8, <pyspark.resultiterable.ResultIterable at 0x726433c03650>),
 (9, <pyspark.resultiterable.ResultIterable at 0x7264932ad3d0>),
 (14, <pyspark.resultiterable.ResultIterable at 0x726433c20310>),
 (15, <pyspark.resultiterable.ResultIterable at 0x726433c21650>),
 (17, <pyspark.resultiterable.ResultIterable at 0x726481b53c10>),
 (18, <pyspark.resultiterable.ResultIterable at 0x726481b52650>),
 (19, <pyspark.resultiterable.ResultIterable at 0x726433c22a50>),
 (20, <pyspark.resultiterable.ResultIterable at 0x726481b3aed0>)]

In [21]:
roots.map(lambda x: (x, x)).join(
    tc_rdd.groupByKey().map(lambda x: (x[0], sorted(x[1])))
).map(lambda x: x[1]).collect()

[(2, [6, 9, 11, 12]),
 (3, [6, 9, 11, 12]),
 (4, [7, 9, 10, 11, 12]),
 (14, [15, 16])]

Arrêter la session Spark.

In [26]:
spark.stop()